In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib as plt
from pydataset import data
import prepare

In [ ]:
df_iris = data('iris')
df_iris.head()

# 4) Iris Data

## print the first 3 rows

In [ ]:
df_iris.head(3)

## print the number of rows and columns (shape)

In [ ]:
df_iris.shape

## print the column names

In [ ]:
df_iris.columns

## print the data type of each column

In [ ]:
df_iris.dtypes

## print the summary statistics for each of the numeric variables. Would you recommend rescaling the data based on these statistics

In [ ]:
df_iris.describe() # No Recommendation to rescale

# 5) Table1_CustDetails table from the Excel_Exercises.xlsx file into a dataframe named df_excel.

In [ ]:
df_excel = pd.read_excel('telco_churn.xlsx')
df_excel

## assign the first 100 rows to a new dataframe, df_excel_sample

In [ ]:
df_excel_sample = df_excel.head(100)
df_excel_sample.shape

## print the number of rows of your original dataframe

In [ ]:
df_excel.shape[0]

## print the first 5 column names

In [ ]:
df_excel.columns[0:5]

## print the column names that have a data type of object

In [ ]:
df_excel.select_dtypes(include='object').columns


## compute the range for each of the numeric variables.

In [ ]:
df_excel.monthly_charges.max() - df_excel.monthly_charges.min()

In [ ]:
df_excel.total_charges.max() - df_excel.total_charges.min()

# 6) Read the data from this google sheet into a dataframe, df_google

In [ ]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'

In [ ]:
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
csv_export_url

In [ ]:
df_googlesheet = pd.read_csv(csv_export_url)

In [ ]:
df_googlesheet.head()

## print the first 3 rows

In [ ]:
df_googlesheet.head(3)

## print the number of rows and columns

In [ ]:
df_googlesheet.shape

## print the column names

In [ ]:
df_googlesheet.columns

## print the data type of each column

In [ ]:
df_googlesheet.dtypes

## print the summary statistics for each of the numeric variables

In [ ]:
df_googlesheet.describe()

## print the unique values for each of your categorical variables

In [ ]:
for col in df_googlesheet.columns:
    if df_googlesheet[col].dtypes == 'object':
        print(f'{col} has {df_googlesheet[col].nunique()} unique values')

In [ ]:
df_googlesheet.Sex.value_counts()

In [ ]:
df_googlesheet.Pclass.value_counts()

In [ ]:
df_googlesheet.Survived.value_counts()

# Data Aquisition Functions -> acquire.py

## 1) Make a function named get_titanic_data that returns the titanic data from the codeup data science database as a pandas data frame. Obtain your data from the Codeup Data Science Database.

In [ ]:
def get_connection(db_name):
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [ ]:
df_titanic = pd.read_sql('SELECT * FROM passengers', get_connection('titanic_db'))
df_titanic.head()

In [ ]:
def get_titanic_data():
    return pd.read_sql('SELECT * FROM passengers', get_connection('titanic_db'))

## 2) Make a function named get_iris_data that returns the data from the iris_db on the codeup data science database as a pandas data frame. The returned data frame should include the actual name of the species in addition to the species_ids. Obtain your data from the Codeup Data Science Database.

In [ ]:
df_iris = pd.read_sql('SELECT * FROM species JOIN measurements USING (species_id)', get_connection('iris_db'))
df_iris.head()

In [ ]:
def get_iris_data():
    return pd.read_sql('SELECT * FROM species JOIN measurements USING (species_id)', get_connection('iris_db'))

## 3) Make a function named get_telco_data

In [ ]:
df_telco = pd.read_sql('''SELECT * FROM customers c
                       JOIN contract_types ct ON ct.contract_type_id=c.contract_type_id
                       JOIN internet_service_types ist ON ist.internet_service_type_id=c.internet_service_type_id
                       JOIN payment_types pt ON pt.payment_type_id=c.payment_type_id
                       '''
                       , get_connection('telco_churn'))
df_telco.head()

In [ ]:
def get_telco_data():
    sql = '''SELECT * FROM customers c
                       JOIN contract_types ct ON ct.contract_type_id=c.contract_type_id
                       JOIN internet_service_types ist ON ist.internet_service_type_id=c.internet_service_type_id
                       JOIN payment_types pt ON pt.payment_type_id=c.payment_type_id
                       '''
    url = get_connection('telco_churn')
    return pd.read_sql(sql, url)

# 4) Add caching to Functions. 

In [ ]:
def get_titanic_data():
    import os
    file_name = 'titanic.csv'
    if os.path.isfile(file_name): # checks if file exist
        df = pd.read_csv(file_name) # assigns df to read on return
    else:
        sql = '''
        'SELECT * FROM passengers'
        '''
        url = get_connection('titanic_db')
        df = pd.read_sql(sql, url) #creates df
        df.to_csv(file_name) # converts df to csv file
    return df

In [ ]:
get_titanic_data().head()

In [ ]:
def get_iris_data():
    import os
    file_name = 'iris.csv'
    if os.path.isfile(file_name):
        df = pd.read_csv(file_name)
    else:
        sql = '''
        SELECT * FROM species 
        JOIN measurements USING (species_id)
        '''
        url = get_connection('iris_db')
        df = pd.read_sql(sql,url)
        df.to_csv(file_name)
    return df

In [ ]:
get_iris_data().head()

In [ ]:
def get_telco_data():
    import os
    file_name = 'telco.csv'
    if os.path.isfile(file_name):
        df = pd.read_csv(file_name)
    else:
        sql = '''
        SELECT * FROM customers c
        JOIN contract_types ct ON ct.contract_type_id=c.contract_type_id
        JOIN internet_service_types ist ON ist.internet_service_type_id=c.internet_service_type_id
        JOIN payment_types pt ON pt.payment_type_id=c.payment_type_id
        '''
        url = get_connection('telco_churn')
        df = pd.read_sql(sql, url)
        df.to_csv(file_name)
    return df

In [ ]:
get_telco_data().head()

# Data Prep Exercises

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# import our own acquire moduleimport acquire

## Iris Data:

### 1) Use the function defined in acquire.py to load the iris data.

In [ ]:
import acquire
df = acquire.get_iris_data()
df.head()

### 2) Drop the species_id and measurement_id columns.

In [ ]:
df = df.drop(columns=['species_id','measurement_id'])
df

### 3) Rename the species_name column to just species.

In [ ]:
df = df.rename(columns={'species_name':'species'})
df.head()

### 4) Create dummy variables of the species name and concatenate onto the iris dataframe

In [ ]:
df_species = pd.get_dummies(df.species, prefix='is')
df = pd.concat([df, df_species], axis=1)
df

### 5) Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

In [ ]:
def prep_iris():
    df = acquire.get_iris_data()
    df = df.drop(columns=['species_id','measurement_id'])
    df = df.rename(columns={'species_name':'species'})
    df_species = pd.get_dummies(df.species, prefix='is')
    df = pd.concat([df, df_species], axis=1)
    return df

# Titanic Data

## 1) Use the function defined in acquire.py to load the Titanic data.

In [ ]:
import acquire
df = acquire.get_titanic_data()
df.head()

## 2) Drop any unnecessary, unhelpful, or duplicated columns.

In [ ]:
drop_col = ['embarked','fare', 'class', 'deck', 'age']
df = df.drop(columns = drop_col)
df.head()

## 3) Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe

In [ ]:
df.info()

In [ ]:
obj_cols = df.columns[[df[col].dtypes == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True, dropna=False))
    print('-------------------------------')

In [ ]:
df['embark_town'] = df.embark_town.fillna(value='Southampton')
df.info()

In [ ]:
dummies_df = pd.get_dummies(df, columns = ['sex', 'embark_town'], drop_first=[True,True])
dummies_df.info()

In [ ]:
dummies_df.head()

In [ ]:
df = pd.concat([df,dummies_df], axis=1)
df.head()

## 4) Create a function named prep_titanic that accepts the raw titanic data, and returns the data with the transformations above applied.

In [ ]:
def prep_titanic():
    df = acquire.get_titanic_data()
    drop_col = ['embarked','fare', 'class', 'deck', 'age']
    df = df.drop(columns = drop_col)
    df['embark_town'] = df.embark_town.fillna(value='Southampton')
    dummies_df = pd.get_dummies(df, columns = ['sex', 'embark_town'], drop_first=[True,True])
    df = pd.concat([df,dummies_df], axis=1)
    return df
    

# Telco Data

## 1) Use the function defined in acquire.py to load the Telco data.

In [ ]:
import acquire
df = acquire.get_telco_data()
df.head()

## 2) Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.duplicated().any()

In [ ]:
# check if columns are duplicated
df.duplicated(subset=['internet_service_type_id','internet_service_type_id.1']).any()

In [ ]:
# check if columns are duplicated
df.duplicated(subset=['contract_type_id','contract_type_id.1']).any()

In [ ]:
# check if columns are duplicated
df.duplicated(subset=['payment_type_id','payment_type_id.1']).any()

In [ ]:
drop_cols = ['internet_service_type_id',
             'internet_service_type_id.1',
             'contract_type_id',
             'contract_type_id.1',
             'payment_type_id',
             'payment_type_id.1']

In [ ]:
df = df.drop(columns=drop_cols)
df.shape

In [ ]:
df.info()

In [ ]:
# shows 11 blank values
df.total_charges.value_counts()

In [ ]:
# show no na values
df[df.total_charges.isna()]

In [ ]:
# shows no null values
df[df.total_charges.isnull()]

In [ ]:
# show total_charges with blank values
df[df.total_charges == ' '].head()

In [ ]:
# replace blank spaces with zeros
df['total_charges'] = df.total_charges.replace(' ', '0')

In [ ]:
# confirmed no more blank values
df[df.total_charges == ' ']

In [ ]:
# convert total charges from object to float
df.total_charges = df.total_charges.astype(float)

In [ ]:
# confirmed change to float
df.info()

In [ ]:
obj_cols = df.columns[[df[col].dtypes == 'O' for col in df.columns]]
obj_cols

In [ ]:
for col in obj_cols:
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True, dropna=False))
    print('-------------------------------------')

## 3) Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe.

In [ ]:
df.columns.to_list()

In [ ]:
# identified columns to encode and assigned to variable
dummies_cols = [
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [ ]:
# performed get_dummmies function to create dummy variables
dummies_df = pd.get_dummies(df[dummies_cols], drop_first=True)
dummies_df.columns.to_list()

In [ ]:
# concat fd with new dummy variable columns
df = pd.concat([df, dummies_df], axis=1)
df.columns.to_list()

## 4) Create a function named prep_telco that accepts the raw telco data, and returns the data with the transformations above applied.

In [ ]:
def prep_telco():
    df = acquire.get_telco_data()
    # id cols to drop
    drop_cols = ['internet_service_type_id',
             'internet_service_type_id.1',
             'contract_type_id',
             'contract_type_id.1',
             'payment_type_id',
             'payment_type_id.1']
    # drop columns
    df = df.drop(columns=drop_cols)
    # replace blank spaces with zeros
    df['total_charges'] = df.total_charges.replace(' ', '0')
    # convert total charges from object to float
    df.total_charges = df.total_charges.astype(float)
    # id columns for dummy variables
    dummies_cols = [
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']
    # creaet dummy variables and assign to variable
    dummies_df = pd.get_dummies(df[dummies_cols], drop_first=True)
    # concat df with dummy variable columns
    df = pd.concat([df, dummies_df], axis=1)
    return df
    

In [ ]:
prep_telco()

In [ ]:
### created prepare.py file

# Split Data

In [11]:
from sklearn.model_selection import train_test_split

In [6]:
df = prepare.prep_titanic()
df.head()

,passenger_id,survived,pclass,sex,sibsp,parch,embark_town,alone,passenger_id,survived,pclass,sibsp,parch,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,3,male,1,0,Southampton,0,0,0,3,1,0,0,1,0,1
1,1,1,1,female,1,0,Cherbourg,0,1,1,1,1,0,0,0,0,0
2,2,1,3,female,0,0,Southampton,1,2,1,3,0,0,1,0,0,1
3,3,1,1,female,1,0,Southampton,0,3,1,1,1,0,0,0,0,1
4,4,0,3,male,0,0,Southampton,1,4,0,3,0,0,1,1,0,1


In [12]:
def split_titanic(df):
    train_validate, test = train_test_split(df, 
                                             test_size=.2, 
                                             random_state=123, 
                                             stratify=df.survived)
    train, validate = train_test_split(train_validate,
                                      test_size=.3,
                                      random_state=123,
                                      stratify=train_validate.survived)
    return train, validate, test

In [13]:
# assign split portions to variables for print
train, validate, test = split_titanic(df)

In [14]:
# print to validate the split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')


train -> (498, 17)
validate -> (214, 17)
test -> (179, 17)


In [16]:
df = prepare.prep_iris()
df

,species,sepal_length,sepal_width,petal_length,petal_width,is_setosa,is_versicolor,is_virginica
0,setosa,5.1,3.5,1.4,0.2,1,0,0
1,setosa,4.9,3.0,1.4,0.2,1,0,0
2,setosa,4.7,3.2,1.3,0.2,1,0,0
3,setosa,4.6,3.1,1.5,0.2,1,0,0
4,setosa,5.0,3.6,1.4,0.2,1,0,0
5,setosa,5.4,3.9,1.7,0.4,1,0,0
6,setosa,4.6,3.4,1.4,0.3,1,0,0
7,setosa,5.0,3.4,1.5,0.2,1,0,0
8,setosa,4.4,2.9,1.4,0.2,1,0,0
9,setosa,4.9,3.1,1.5,0.1,1,0,0


In [17]:
def split_iris(df):
    train_validate, test = train_test_split(df, 
                                             test_size=.2, 
                                             random_state=123, 
                                             stratify=df.species)
    train, validate = train_test_split(train_validate,
                                      test_size=.3,
                                      random_state=123,
                                      stratify=train_validate.species)
    return train, validate, test

In [18]:
# assign split portions to variables for print
train, validate, test = split_iris(df)

In [19]:
# print to validate the split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (84, 8)
validate -> (36, 8)
test -> (30, 8)


In [20]:
df = prepare.prep_telco()
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,...,1,1,0,0,1,0,0,0,0,1
1,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,...,0,0,0,0,1,0,0,1,0,0
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,...,0,0,0,0,1,0,0,0,0,0
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,...,0,1,0,0,1,0,0,1,0,0
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,...,1,1,0,0,1,0,0,0,0,0


In [21]:
def split_telco(df):
    train_validate, test = train_test_split(df, 
                                             test_size=.2, 
                                             random_state=123, 
                                             stratify=df.churn)
    train, validate = train_test_split(train_validate,
                                      test_size=.3,
                                      random_state=123,
                                      stratify=train_validate.churn)
    return train, validate, test

In [22]:
# assign split portions to variables for print
train, validate, test = split_telco(df)

In [23]:
# print to validate the split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (3943, 49)
validate -> (1691, 49)
test -> (1409, 49)
